<a href="https://colab.research.google.com/github/karelcuevas/Actividades_Aprendizaje-/blob/main/%C3%81rboles_de_clasificaci%C3%B3n_Equipo20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Paso 1: Importar bibliotecas y establecer la conexión con Spark
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName('DecisionTree').getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Paso 2: Leer dataset

import pandas as pd

df = pd.read_csv('/content/sparkdata2008.csv')
df

,cve_mun,nom_estab,per_ocu,region,indicador,valor,unidad_medida,ano,id_indicador
0,120,ACUACULTORES LOS CASTRO S.C. DE R.L. DE C.V.,0 a 5 personas,centro,Personal ocupado total.,282211.0,Número de personas,2008.0,5.300000e+09
1,120,AGROSERVICIOS DINAMICOS INTEGRALES,0 a 5 personas,centro,Personal ocupado total.,282211.0,Número de personas,2008.0,5.300000e+09
2,120,AGROSERVICIOS DINAMICOS INTEGRALES,6 a 10 personas,centro,Personal ocupado total.,282211.0,Número de personas,2008.0,5.300000e+09
3,120,AMBIENT,0 a 5 personas,centro,Personal ocupado total.,282211.0,Número de personas,2008.0,5.300000e+09
4,120,DESHUESADERO DE COCO,11 a 30 personas,centro,Personal ocupado total.,282211.0,Número de personas,2008.0,5.300000e+09
...,...,...,...,...,...,...,...,...,...
378434,4,VIDRIOS ALUMINIOS Y HERRERIA PACOS,0 a 5 personas,sur,Personal ocupado total.,246.0,Número de personas,2008.0,5.300000e+09
378435,4,PRECIDENCIA MUNICIPAL. AYUNTAMIENTO.,101 a 250 personas,sur,Personal ocupado total.,246.0,Número de personas,2008.0,5.300000e+09
378436,4,PROTECCION CIVIL,6 a 10 personas,sur,Personal ocupado total.,246.0,Número de personas,2008.0,5.300000e+09
378437,4,RECAUDADORA 121 SECRETARIA DE FINANZAS DE AMAC...,0 a 5 personas,sur,Personal ocupado total.,246.0,Número de personas,2008.0,5.300000e+09


In [ ]:
# Paso 3: Transformar datos

from pyspark.sql.types import StructType,StructField, StringType, IntegerType

mySchema = StructType([ StructField("cve_num", IntegerType(), True)\
                       ,StructField("nom_estab", StringType(), True)\
                       ,StructField("per_ocu", StringType(), True)\
                       ,StructField("region", StringType(), True)\
                       ,StructField("indicador", StringType(), True)\
                       ,StructField("valor", IntegerType(), True)\
                       ,StructField("unidad_medida", StringType(), True)\
                       ,StructField("ano", IntegerType(), True)\
                       ,StructField("id_indicador", IntegerType(), True)
                       ])


In [ ]:
# Paso 4: Limpiar datos y convertir a numéricas las columnas region_num y per_ocu_num

df['cve_mun'] = pd.to_numeric(df['cve_mun'], downcast="integer", errors = 'coerce')
df['valor'] = pd.to_numeric(df['valor'], downcast="integer", errors = 'coerce')
df['ano'] = pd.to_numeric(df['ano'], downcast="integer", errors = 'coerce')
df['id_indicador'] = pd.to_numeric(df['id_indicador'], downcast="integer", errors = 'coerce')

df['region_num'] = df['region'].map({x:y for x,y in zip(df['region'].unique(), range(1,20))})
df['per_ocu_num'] = df['per_ocu'].map({x:y for x,y in zip(df['per_ocu'].unique(), range(1,20))})

In [ ]:
# Paso 5: Crear un DataFrame de Spark

spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkDF=spark.createDataFrame(df) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- cve_mun: byte (nullable = true)
 |-- nom_estab: string (nullable = true)
 |-- per_ocu: string (nullable = true)
 |-- region: string (nullable = true)
 |-- indicador: string (nullable = true)
 |-- valor: integer (nullable = true)
 |-- unidad_medida: string (nullable = true)
 |-- ano: short (nullable = true)
 |-- id_indicador: long (nullable = true)
 |-- region_num: long (nullable = true)
 |-- per_ocu_num: long (nullable = true)

+-------+--------------------+----------------+------+--------------------+------+------------------+----+------------+----------+-----------+
|cve_mun|           nom_estab|         per_ocu|region|           indicador| valor|     unidad_medida| ano|id_indicador|region_num|per_ocu_num|
+-------+--------------------+----------------+------+--------------------+------+------------------+----+------------+----------+-----------+
|    120|ACUACULTORES LOS ...|  0 a 5 personas|centro|Personal ocupado ...|282211|Número de personas|2008|  5300000011|         1

In [121]:
# Paso 6: Convertir la variable categórica en una variable numérica y crear el set de features

indexer = StringIndexer(inputCol='region_num', outputCol='Región')
indexed = indexer.fit(sparkDF).transform(sparkDF)

assembler = VectorAssembler(inputCols=['per_ocu_num', 'valor', 'ano'], outputCol='features')
assembled = assembler.transform(indexed)

In [122]:
# Paso 7: Dividir los datos en conjuntos de entrenamiento y prueba

(trainingData, testData) = assembled.randomSplit([0.7, 0.3], seed=1234)

In [123]:
# Paso 8: Crear un modelo de árbol de decisión con el método DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol='Región', featuresCol='features')

In [124]:
# Paso 9: Ajustar el modelo a los datos de entrenamiento y efectuar las predicciones de los datos 

model = dt.fit(trainingData)
predictions = model.transform(testData)

In [125]:
# Paso 10: Evaluar el rendimiento del modelo

evaluator = MulticlassClassificationEvaluator(labelCol='Región', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

print('Accuracy:', accuracy)

Accuracy: 0.7270429803478015
